In [32]:
import pandas as pd

data = pd.read_csv("/home/ictc/Desktop/078msdsa016/covid_sentiment/CovidNepaliTweet_Research/sentiment_youtube_comment/collected_final_data.csv")

In [33]:
import torch
import numpy as np 
import pandas as pd 
from transformers import BertTokenizer, BertModel, BertForMaskedLM, AutoTokenizer, AutoModelForMaskedLM
from scipy.spatial.distance import cosine 
import tokenizers 
import pandas as pd 
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix, f1_score, accuracy_score
from nltk.corpus import stopwords
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
# import snowballstemmer
import numpy
import os 
import re
import json
import pickle 

model = AutoModelForMaskedLM.from_pretrained("Rajan/NepaliBERT", output_hidden_states = True, return_dict = True, output_attentions = True)

tokenizers = AutoTokenizer.from_pretrained("Rajan/NepaliBERT")

In [34]:
def get_bert_embedding_sentence(input_sentence):
    md = model
    tokenizer = tokenizers
#     md = local_model
#     tokenizer = local_tokenizers
    marked_text = " [CLS] " + input_sentence + " [SEP] "
    tokenized_text = tokenizer.tokenize(marked_text)

    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1] * len(indexed_tokens) 
    
    # Convert inputs to Pytorch tensors
    tokens_tensors = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    
    with torch.no_grad():
        outputs = md(tokens_tensors, segments_tensors)
        # removing the first hidden state
        # the first state is the input state 

        hidden_states = outputs.hidden_states
#         print(hidden_states[-2])
        # second_hidden_states = outputs[2]
    # hidden_states has shape [13 x 1 x 22 x 768]

    # token_vecs is a tensor with shape [22 x 768]
#     token_vecs = hidden_states[-2][0]
    # get last four layers
#     last_four_layers = [hidden_states[i] for i in (-1,-2, -3,-4)]
    # cast layers to a tuple and concatenate over the last dimension
#     cat_hidden_states = torch.cat(tuple(last_four_layers), dim=-1)
#     print(cat_hidden_states.shape)
#     import pdb;pdb.set_trace()
    token_vecs = hidden_states[-2][0]
    # take the mean of the concatenated vector over the token dimension
#     sentence_embedding = torch.mean(cat_hidden_states, dim=0).squeeze()

    # Calculate the average of all 22 token vectors.
    sentence_embedding = torch.mean(token_vecs, dim=0)
#     sentence_embedding = torch.mean(token_vecs, dim=1)
    return sentence_embedding.numpy()

In [35]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('Rajan/NepaliBERT')

def get_bert_embedding_sentence(sentence):

    query_embedding = model.encode(sentence)

    return query_embedding

No sentence-transformers model found with name /home/ictc/.cache/torch/sentence_transformers/Rajan_NepaliBERT. Creating a new one with MEAN pooling.
Some weights of BertModel were not initialized from the model checkpoint at /home/ictc/.cache/torch/sentence_transformers/Rajan_NepaliBERT and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
data['word_embeddings'] = data['text'].apply(get_bert_embedding_sentence)

In [37]:
data.to_csv("embeddings_data_nepali_BERT.csv")

In [38]:
X,y = data['word_embeddings'], data['label']

In [39]:
train_X, test_X, train_y, test_y = train_test_split(X,y, test_size = 0.2, random_state = 420)

In [40]:
svc = SVC(probability=True)

In [41]:
svc.fit(train_X.tolist(), train_y)
#svc.fit(train_X, train_y)

SVC(probability=True)

In [42]:
svc_pred = svc.predict(test_X.tolist())
# svc_pred = svc.predict(test_X)

In [43]:
print(confusion_matrix(test_y, svc_pred))

[[391 124]
 [ 90 607]]


In [44]:
print(classification_report(test_y, svc_pred))

              precision    recall  f1-score   support

           0       0.81      0.76      0.79       515
           1       0.83      0.87      0.85       697

    accuracy                           0.82      1212
   macro avg       0.82      0.82      0.82      1212
weighted avg       0.82      0.82      0.82      1212



In [45]:
accuracy_score(test_y, svc_pred)

0.8234323432343235

In [46]:
from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier(max_depth=2, random_state=0)
random_forest.fit(train_X.tolist(), train_y)



RandomForestClassifier(max_depth=2, random_state=0)

In [47]:
random_pred = random_forest.predict(test_X.tolist())
print(accuracy_score(test_y, svc_pred))

print(confusion_matrix(test_y, random_pred))

print(classification_report(test_y, random_pred))

0.8234323432343235
[[106 409]
 [ 13 684]]
              precision    recall  f1-score   support

           0       0.89      0.21      0.33       515
           1       0.63      0.98      0.76       697

    accuracy                           0.65      1212
   macro avg       0.76      0.59      0.55      1212
weighted avg       0.74      0.65      0.58      1212



# Naive Bayes

In [48]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(train_X.tolist(), train_y)


GaussianNB()

In [49]:
gnb_pred = gnb.predict(test_X.tolist())
print(accuracy_score(test_y, gnb_pred))

print(confusion_matrix(test_y, gnb_pred))

print(classification_report(test_y, gnb_pred))

0.7277227722772277
[[406 109]
 [221 476]]
              precision    recall  f1-score   support

           0       0.65      0.79      0.71       515
           1       0.81      0.68      0.74       697

    accuracy                           0.73      1212
   macro avg       0.73      0.74      0.73      1212
weighted avg       0.74      0.73      0.73      1212



# Ada Boost

In [50]:
from sklearn.ensemble import AdaBoostClassifier
ada_boost_clf = AdaBoostClassifier(n_estimators=100, random_state=0)
ada_boost_clf.fit(train_X.tolist(), train_y)


AdaBoostClassifier(n_estimators=100, random_state=0)

In [51]:
adaboost_pred = ada_boost_clf.predict(test_X.tolist())
print(accuracy_score(test_y, adaboost_pred))

print(confusion_matrix(test_y, adaboost_pred))

print(classification_report(test_y, adaboost_pred))

0.7929042904290429
[[377 138]
 [113 584]]
              precision    recall  f1-score   support

           0       0.77      0.73      0.75       515
           1       0.81      0.84      0.82       697

    accuracy                           0.79      1212
   macro avg       0.79      0.78      0.79      1212
weighted avg       0.79      0.79      0.79      1212



# Decision Tree

In [52]:
from sklearn.tree import DecisionTreeClassifier
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=2)
decision_tree.fit(train_X.tolist(), train_y)


DecisionTreeClassifier(max_depth=2, random_state=0)

In [53]:
decision_pred = decision_tree.predict(test_X.tolist())
print(accuracy_score(test_y, decision_pred))

print(confusion_matrix(test_y, decision_pred))

print(classification_report(test_y, decision_pred))

0.6782178217821783
[[279 236]
 [154 543]]
              precision    recall  f1-score   support

           0       0.64      0.54      0.59       515
           1       0.70      0.78      0.74       697

    accuracy                           0.68      1212
   macro avg       0.67      0.66      0.66      1212
weighted avg       0.67      0.68      0.67      1212



# Extra Trees Classifier

In [54]:
from sklearn.ensemble import ExtraTreesClassifier
etc_clf = ExtraTreesClassifier(n_estimators=100, random_state=0)
etc_clf.fit(train_X.tolist(), train_y)


ExtraTreesClassifier(random_state=0)

In [55]:
etc_pred = etc_clf.predict(test_X.tolist())
print(accuracy_score(test_y, etc_pred))

print(confusion_matrix(test_y, etc_pred))

print(classification_report(test_y, etc_pred))

0.8217821782178217
[[362 153]
 [ 63 634]]
              precision    recall  f1-score   support

           0       0.85      0.70      0.77       515
           1       0.81      0.91      0.85       697

    accuracy                           0.82      1212
   macro avg       0.83      0.81      0.81      1212
weighted avg       0.83      0.82      0.82      1212



# Logistic Regression

In [56]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(C=1e5)
logreg.fit(train_X.tolist(), train_y)

/home/ictc/anaconda3/envs/shushanta_nepali_gpt/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=100000.0)

In [57]:
lr_pred = logreg.predict(test_X.tolist())
print(accuracy_score(test_y, lr_pred))

print(confusion_matrix(test_y, lr_pred))

print(classification_report(test_y, lr_pred))

0.8168316831683168
[[388 127]
 [ 95 602]]
              precision    recall  f1-score   support

           0       0.80      0.75      0.78       515
           1       0.83      0.86      0.84       697

    accuracy                           0.82      1212
   macro avg       0.81      0.81      0.81      1212
weighted avg       0.82      0.82      0.82      1212



# KNN

In [58]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(train_X.tolist(), train_y)

KNeighborsClassifier(n_neighbors=3)

In [59]:
neigh_pred = neigh.predict(test_X.tolist())
print(accuracy_score(test_y, neigh_pred))

print(confusion_matrix(test_y, neigh_pred))

print(classification_report(test_y, neigh_pred))

0.7722772277227723
[[335 180]
 [ 96 601]]
              precision    recall  f1-score   support

           0       0.78      0.65      0.71       515
           1       0.77      0.86      0.81       697

    accuracy                           0.77      1212
   macro avg       0.77      0.76      0.76      1212
weighted avg       0.77      0.77      0.77      1212



# MLP

In [60]:
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(random_state=1, max_iter=300)
mlp.fit(train_X.tolist(), train_y)

MLPClassifier(max_iter=300, random_state=1)

In [61]:
mlp_pred = mlp.predict(test_X.tolist())
print(accuracy_score(test_y, mlp_pred))

print(confusion_matrix(test_y, mlp_pred))

print(classification_report(test_y, mlp_pred))

0.8473597359735974
[[428  87]
 [ 98 599]]
              precision    recall  f1-score   support

           0       0.81      0.83      0.82       515
           1       0.87      0.86      0.87       697

    accuracy                           0.85      1212
   macro avg       0.84      0.85      0.84      1212
weighted avg       0.85      0.85      0.85      1212

